Tiger/line: census geospatial data
==================================
The  US Census Beureau provides geospatial "shapefiles"
that correspond to the statistical data the collect
and make available through the API.

In this notebook we will demonstrate how to connect to these shapefiles
using `geopandas` and how to **merge** them with the statistical data
to create interative `folium` maps.


**Goals:**

- set up and test out your own Census API key
- understand some key geographic levels in the Census API
- work with Census fields and geographies to query data

**To follow this lab, you will have to [sign up for a Census API key](https://api.census.gov/data/key_signup.html).** You will need your key to run the code in this notebook. Students in CSC 602 can put **Adelphi University** as the organization.

[Watch the video walkthrough](https://youtu.be/AwyhtIcpeLw) [35:11]

Setup
=====
- loading libraries
- initializing your API key



Census API Data
===============
We will load basic demographic data from the Census API.
See [lab 7](https://drive.google.com/file/d/1SxMqPCqJvNwL4U40bWYWDWF8u5rqAVfF/view?usp=sharing) for more details on how to use the Census API.

The US States
=============



In [1]:
# install libraries that are not part of Colab by default
!pip install census us mapclassify -q


%load_ext autoreload
from maptools import tiger, census_vars

# load the libraries we need
from census import Census
import us
import pandas as pd
import geopandas as gpd

# replace this with your own Census API key (this is a fake key)
api_key = "796f9e16b6e3f73329d0d36de60d226d53215cc5"
root = "/home/mxc/Projects/cartopy/tiger"

In [2]:
import os
api_key = os.getenv('CENSUS_API_KEY')


# from google.colab import userdata
# api_key = userdata.get('CENSUS_API_KEY')

api_key
gpd

<module 'geopandas' from '/home/mxc/.virtualenvs/cartopy/lib/python3.10/site-packages/geopandas/__init__.py'>

In [8]:
# places
places = "https://www2.census.gov/geo/tiger/TIGER2024/PLACE/"
t = tiger.load_tiger_dir(url=places, root=root)

path = os.path.join(root,"PLACE")
print(path)
out = os.path.join(root,"places")
# walk path and move all .geojson files to out
for r, dirs, files in os.walk(path):
    for f in files:
        if f.endswith(".geojson"):
            # tl_2024_01_place.geojson
            state = f.split("_")[2]
            newname = census_vars.lookup_state(state).lower() + ".geojson"
            og = os.path.join(r, f)
            os.rename(og, os.path.join(out, newname))

t = [os.path.join(out, f) for f in os.listdir(out) if f.endswith(".geojson")]

df = pd.concat([gpd.read_file(f) for f in t])
df.to_file(os.path.join(root, "places/places.geojson"), driver="GeoJSON")


In [26]:
# landmarks
landmarks = "https://www2.census.gov/geo/tiger/TIGER2024/POINTLM/"
t = tiger.load_tiger_dir(url=landmarks, root=root)

path = os.path.join(root, "POINTLM")
print(path)
out = os.path.join(root,"landmarks")
# walk path and move all .geojson files to out
for r, dirs, files in os.walk(path):
    for f in files:
        if f.endswith(".geojson"):
            # tl_2024_01_place.geojson
            state = f.split("_")[2]
            newname = census_vars.lookup_state(state).lower() + ".geojson"
            og = os.path.join(r, f)
            os.rename(og, os.path.join(out, newname))

t = [os.path.join(out, f) for f in os.listdir(out) if f.endswith(".geojson")]

df = pd.concat([gpd.read_file(f) for f in t])
df.to_file(os.path.join(root, "landmarks/landmarks.geojson"), driver="GeoJSON")


In [33]:
# tracts
url = "https://www2.census.gov/geo/tiger/TIGER2024/TRACT/"
t = tiger.load_tiger_dir(url=url, root=root)
out = os.path.join(root, "tracts")
os.makedirs(out, exist_ok=True)
df = pd.concat([gpd.read_file(f) for f in t])
df.to_file(os.path.join(out, "tracts.geojson"), driver="GeoJSON")

for og in t:
    f = os.path.basename(og)
    state = f.split("_")[2]
    newname = census_vars.lookup_state(state).lower() + ".geojson"
    os.rename(og, os.path.join(out, newname))

In [34]:
# school districts
url = "https://www2.census.gov/geo/tiger/TIGER2024/UNSD/"
t = tiger.load_tiger_dir(url=url, root=root)
out = os.path.join(root, "schools")
os.makedirs(out, exist_ok=True)
df = pd.concat([gpd.read_file(f) for f in t])
df.to_file(os.path.join(out, "schools.geojson"), driver="GeoJSON")

for og in t:
    f = os.path.basename(og)
    state = f.split("_")[2]
    newname = census_vars.lookup_state(state).lower() + ".geojson"
    os.rename(og, os.path.join(out, newname))


In [35]:
# zip codes
url = "https://www2.census.gov/geo/tiger/TIGER2024/ZCTA520/"
t = tiger.load_tiger_dir(url=url, root=root)
df = gpd.read_file( "/home/mxc/Projects/cartopy/tiger/ZCTA520/tl_2024_us_zcta520/tl_2024_us_zcta520.geojson")
df
out = os.path.join(root, "zipcodes")
os.makedirs(out, exist_ok=True)

states = gpd.read_file(os.path.join(root, "states.geojson"))

def state_zips(state):
    x = df[df.geometry.intersects(state.geometry)]
    x.to_file(os.path.join( out, f"{state.STATE.lower()}.geojson"), driver="GeoJSON")


states["STATE"] = states.STATEFP.apply(census_vars.lookup_state)
_ = states.apply(state_zips, axis=1)
os.rename("/home/mxc/Projects/cartopy/tiger/ZCTA520/tl_2024_us_zcta520/tl_2024_us_zcta520.geojson", os.path.join(out, "zipcodes.geojson"))

['/home/mxc/Projects/cartopy/tiger/ZCTA520/tl_2024_us_zcta520/tl_2024_us_zcta520.geojson']

In [ ]:

# consolidated cities
concity = "https://www2.census.gov/geo/tiger/TIGER2024/CONCITY/"
t = tiger.load_tiger_dir(concity, root=root)
df = pd.concat([gpd.read_file(f) for f in t])
df.to_file(root + "/concity.shp")

In [3]:
# counties
counties = gpd.read_file( "/home/mxc/Projects/cartopy/tiger/tmp/tl_2024_us_county/tl_2024_us_county.shp")


In [6]:

# counties by state
counties["STATE"] = counties["STATEFP"].apply(census_vars.lookup_state)
def state_shapes(state):
    x = counties[counties["STATE"] == state]
    x.to_file(os.path.join(root, f"counties/{state.lower()}.geojson"), driver="GeoJSON")

counties.STATE.apply(state_shapes)
counties.to_file(os.path.join(root, "counties/counties.geojson"), driver="GeoJSON")

In [ ]:

# coastline
coast = gpd.read_file( "/home/mxc/Projects/cartopy/tiger/tmp/tl_2024_us_coastline/tl_2024_us_coastline.shp")
coast.to_file(os.path.join(root, "coastline.geojson"), driver="GeoJSON")